# COVID-19 Auswertung - Impfungen


## import libraries

In [1]:
Programm_Name = "Impfungen" # 04.01.2021 - Time Series Impffdaten erstellen

In [2]:
import pandas as pd #Daten
from matplotlib import pyplot as plt # plots
#import matplotlib as mpl
from matplotlib.ticker import FuncFormatter   # Numberformat plot ticks
import numpy as np

# performance
from datetime import datetime
from datetime import date # todays date
#import seaborn as sns

import os
now = datetime.now()

# Variablen für die Plots definieren

### Linien und Schriftgrößen

In [3]:
# Linien Stärke
lws = 3
lwb = 7
# Bezugsschriftgröße
size = 25

 ### Bezugsgröße Einwohner

In [4]:
pE = 100_000

### Speicherorte und Dateinamen Plots

In [5]:
pfad = "D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\"

name_7 = "plot_7_Impfquote_ger.png" # Impfquote Deutschland Bundesländer
name_7_2 = "plot_7-2_Impfungen_Timeseries_ger.png" # Impfungen Timeseries Deutschland
name_7_3 = "plot_7-3_Impfungen_Timeseries_ger.png" # Impfungen Timeseries Deutschland

## Länder Farben

In [6]:
c_fr = '#f80d0d'    # rot
c_pl = '#25e5e5'    # cyan
c_isr = '#25e5e5'
c_ger = '#0721ff'   # dunkelblau
c_cz = '#bb8fce'    # lila
c_usa = '#bb8fce'
c_at = '#18c213'    # grün
c_rus = '#18c213'
c_se = '#18c213'
c_ch = '#000000'    # schwarz
c_ita = '#000000'
c_gb = '#faac2b'    # orange


### Einwohnerzahlen

In [7]:
E_fr = 66_012_908
E_pl = 38_659_927
E_ger = 83_020_000
E_cz = 10_586_651
E_at = 8_902_600
E_ch = 8_847_020
E_gb = 66_650_000

E_isr = 8_884_000
E_usa = 328_200_000
E_rus = 144_500_000
E_ita = 60_360_000
E_spa = 46_940_000
E_se = 10_230_000


# Preprocessing Dataframes

In [8]:
today = date.today().strftime("%d.%m.%Y")

## Datenimport

In [9]:
# Datenimport Einwohnerdaten Bundesländer
df_EW = pd.read_excel("D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\input\\02-bundeslaender.xlsx",
                     sheet_name="Bundesländer_mit_Hauptstädten",
                     usecols="A:G",
                     skiprows=6)


# Impfungen sheet_name der Datenquelle auslesen
df_I_sheet = pd.read_excel("D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\input\\Impfquotenmonitoring.xlsx", None)
sheet_name = df_I_sheet.keys()         # Tabellenblätter Namen auslesen
sheet_name = [*sheet_name][1]          # Namen des zweiten Tabellenblattes anzeigen [*dict] dict keyword->list Index 1 anzeigen


# Datenimport Impfungen
df_I = pd.read_excel("D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\input\\Impfquotenmonitoring.xlsx",
                    sheet_name = sheet_name,
                    usecols="A:I")


## preprocessing

### data (WHO)

In [10]:
#Timestamp definieren  YYYY-MM-DD
ts = pd.to_datetime('2020-10-01', utc=True)

### df_EW (Einwohnerdaten pro Bundesland)

In [11]:
# Spalten Überschriften definieren 
df_EW.rename(columns= {"Unnamed: 0": "Bundesland"},inplace=True)
df_EW.rename(columns= {"Unnamed: 1": "Stadt"},inplace=True)
df_EW.rename(columns= {"Unnamed: 2": "Fläche"},inplace=True)
df_EW.rename(columns= {"Unnamed: 3": "Einwohner"},inplace=True)
df_EW.rename(columns= {"Unnamed: 4": "Einwohner_M"},inplace=True)
df_EW.rename(columns= {"Unnamed: 5": "Einwohner_F"},inplace=True)
df_EW.rename(columns= {"Unnamed: 6": "Einwohner_pro_km²"},inplace=True)

# nicht gebrauchte columns löschen
df_EW = df_EW.drop(columns=['Stadt','Einwohner_M','Einwohner_F','Einwohner_pro_km²'])

# Rechnen
df_EW["Einwohner_pro_km²"] = df_EW["Einwohner"]/df_EW["Fläche"]

# alles löschen was keine Bundesländer sind
df_EW["Bundesland"] = df_EW["Bundesland"].fillna("x")      # ersetze alle NaN Werte mit x
df_EW["Fläche"] = df_EW["Fläche"].fillna("x")              # ersetze alle NaN Werte mit x
indexNames = df_EW[ df_EW['Bundesland'] == "x" ].index     # alle Index Werte mit "x" in eine Variable schreiben
df_EW.drop(indexNames , inplace=True)                      # lösche diese Index rows vom dataframe
indexNames = df_EW[ df_EW['Fläche'] == "x" ].index         # wiederholen
df_EW.drop(indexNames , inplace=True)                      # wiederholen


# Bundesland erste 3 Zeichen entfernen
df_EW["Bundesland"] = df_EW["Bundesland"].str[3:]

# Datentyp in float ändern
df_EW["Fläche"] = pd.to_numeric(df_EW["Fläche"], errors='coerce')

#remove space at start
df_EW["Bundesland"] = df_EW["Bundesland"].str.lstrip()
#df_EW.head(1)

### df_I (Impfdaten pro Bundesland)

In [12]:
df_I.head(1)

,RS,Bundesland,Impfungen kumulativ,Differenz zum Vortag,Impfungen pro 1.000 Einwohner,Indikation nach Alter*,Berufliche Indikation*,Medizinische Indikation*,Pflegeheim-bewohnerIn*
0,08,Baden-Württemberg,76762.0,5869.0,6.91525,41457.0,22519.0,2980.0,12781.0


In [13]:

# nicht gebrauchte columns löschen
df_I = df_I.drop(columns=['Indikation nach Alter*','Berufliche Indikation*','Medizinische Indikation*', 'Pflegeheim-bewohnerIn*'])

# nicht gebrauchte rows löschen
df_I = df_I.dropna(axis=0, how='any')
#df_I.drop([16,17,18,19,20] , inplace=True)

df_I["Differenz zum Vortag"] = df_I["Differenz zum Vortag"].fillna(0)


### df_I_EW (Impfquote)

In [14]:
# #df_I_EW.head(1)

In [15]:
# Timeseries gestern einlesen
df_I_timeseries1 = pd.read_csv('D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_I_timeseries.csv')

In [16]:
# Impfquote in % pro Bundesland
# VLOOKUP 
df_I_EW = pd.merge(df_I, df_EW, on='Bundesland', how='outer')   # how='outer','inner','left','right'
df_I_EW["Impfquote [%]"] = (df_I_EW["Impfungen kumulativ"]/df_I_EW["Einwohner"])*100*(1/2)
df_I_EW.drop(columns= ['Fläche', 'Einwohner', 'Einwohner_pro_km²'])
df_I_EW = df_I_EW.sort_values(by=['Impfquote [%]'], ascending=False)

### Berechnung Herdeinimmunität

In [17]:

Impfstart = datetime.strptime('28.12.2020', '%d.%m.%Y')
Tage_seit_Impfstart = (now-Impfstart).days
Summe_Impfungen = df_I_EW["Impfungen kumulativ"].sum()
Impfgeschwindigkeit = (1/2)*Summe_Impfungen/Tage_seit_Impfstart
Impfgeschwindigkeit_gestern = (1/2)*df_I_EW["Differenz zum Vortag"].sum()

if Impfgeschwindigkeit_gestern > Impfgeschwindigkeit:
    Impfgeschwindigkeit = (Impfgeschwindigkeit*0.2 + Impfgeschwindigkeit_gestern*0.8)
    print(f'Impfgeschwindigkeit = {Impfgeschwindigkeit} IF-Statement')

Herdenimmun = round((1/12)*(1/30)*(E_ger*0.6)*(1/(Impfgeschwindigkeit)),1)   # grobe Schätzung Anzahl Monate bis Herdenimmun



print(f'Tage_seit_Impfstart = {Tage_seit_Impfstart}')   # wie lange schon geimpft wird
print(f'Summe_Impfungen = {Summe_Impfungen}')
print(f'Impfgeschwindigkeit = {Impfgeschwindigkeit} Mittelwert')
print(f'Impfgeschwindigkeit_gestern = {Impfgeschwindigkeit_gestern}')
print(f'E_ger = {E_ger}')   # Einwohner Deutschland
print(f'Herdenimmun = {Herdenimmun} Jahren')

Impfgeschwindigkeit = 32409.28125 IF-Statement
Tage_seit_Impfstart = 16
Summe_Impfungen = 758093.0
Impfgeschwindigkeit = 32409.28125 Mittelwert
Impfgeschwindigkeit_gestern = 34589.0
E_ger = 83020000
Herdenimmun = 4.3 Jahren


In [18]:
# v=s/t    v = 83020000*0.6 / t   t = 

Ende = datetime.strptime('01.10.2021', '%d.%m.%Y')
Tage_bis_Oktober = (Ende - now).days
print(f'Tage_bis_Oktober = {Tage_bis_Oktober} Tage')

Impfungen60 = 2*(E_ger*0.6 - Summe_Impfungen)
SOLL_Impfungen60 = int(Impfungen60*(1/Tage_bis_Oktober))
print(f'SOLL_Impfungen60 = {SOLL_Impfungen60} Impfungen pro Tag')

Impfungen30 = 2*(E_ger*0.3 - Summe_Impfungen)
SOLL_Impfungen30 = int(Impfungen30*(1/Tage_bis_Oktober))
print(f'SOLL_Impfungen30 = {SOLL_Impfungen30} Impfungen pro Tag')

Impfungen15 = 2*(E_ger*0.15 - Summe_Impfungen)
SOLL_Impfungen15 = int(Impfungen15*(1/Tage_bis_Oktober))
print(f'SOLL_Impfungen15 = {SOLL_Impfungen15} Impfungen pro Tag')

Impfungen5 = 2*(E_ger*0.05 - Summe_Impfungen)
SOLL_Impfungen5 = int(Impfungen5*(1/Tage_bis_Oktober))
print(f'SOLL_Impfungen5 = {SOLL_Impfungen5} Impfungen pro Tag')


Tage_bis_Oktober = 260 Tage
SOLL_Impfungen60 = 377337 Impfungen pro Tag
SOLL_Impfungen30 = 185753 Impfungen pro Tag
SOLL_Impfungen15 = 89960 Impfungen pro Tag
SOLL_Impfungen5 = 26099 Impfungen pro Tag


## df_I_TS Impfungen Timeseries

In [19]:
# Impfungen sheet_name der Datenquelle auslesen
df_I_sheet = pd.read_excel("D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\input\\Impfquotenmonitoring.xlsx", None)
sheet_name = df_I_sheet.keys()         # Tabellenblätter Namen auslesen
sheet_name = [*sheet_name][2]          # Namen des zweiten Tabellenblattes anzeigen [*dict] dict keyword->list Index 1 anzeigen

# Datenimport Impfungen
df_I_TS = pd.read_excel("D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\input\\Impfquotenmonitoring.xlsx",
                    sheet_name = sheet_name,
                    usecols="A:B")



#gleitender Mittelwert der cases und deaths, 7 Tage
df_I_TS['MA'] = df_I_TS['Gesamtzahl Impfungen'].rolling(window=7,min_periods=1).mean()

df_I_TS['MSTD_Impfungen'] = df_I_TS['Gesamtzahl Impfungen'].rolling(window=7,min_periods=1).std()

df_I_TS['OTG_Impfungen'] = df_I_TS['MA'] + df_I_TS['MSTD_Impfungen']
df_I_TS['UTG_Impfungen'] = df_I_TS['MA'] - df_I_TS['MSTD_Impfungen']

df_I_TS['SOLL_Impfungen60'] = SOLL_Impfungen60
df_I_TS['SOLL_Impfungen30'] = SOLL_Impfungen30
df_I_TS['SOLL_Impfungen15'] = SOLL_Impfungen15
df_I_TS['SOLL_Impfungen5'] = SOLL_Impfungen5

# Date_reported in eine Datumsspalte umwandeln
#df_I_TS['Datum der Impfung'] = pd.to_datetime(df_I_TS.Date_reported, utc=True)
df_I_TS = df_I_TS.dropna(axis=0, how='any')

df_I_TS = df_I_TS[df_I_TS.Datum != 'Impfungen gesamt']

# Visualisierung der Plots

In [20]:
x = (1/2)*df_I_EW["Bundesland"].count()
y = max(df_I_EW["Impfquote [%]"])*0.87
Impfquote_ger = round((df_I_EW["Impfungen kumulativ"].sum()/E_ger)*100*(1/2),2)

print(f'x = {x}')
print(f'y = {y}')
print(f'Impfquote_ger = {Impfquote_ger} %')

x = 8.0
y = 0.7871525951131059
Impfquote_ger = 0.46 %


## Impfquote - Bundesländer 

In [21]:
plt.figure(figsize=(16,9))

plt.style.use('seaborn')

plt.bar(x=df_I_EW["Bundesland"], height=df_I_EW["Impfquote [%]"], width = 0.9, align = 'center', color = c_ger)

# # Schriftgrößen x und y achsenwerte
plt.xticks(fontsize=size - 10, rotation=90)
plt.yticks(fontsize=size -4)

plt.ylabel('Impfquote [%]', fontsize=size)
plt.xlabel('Bundesland', fontsize=size)

plt.title('Impfquote - Deutschland (RKI-Daten)\n', fontsize=size)
plt.suptitle(today + ' PW', fontsize=size-5, y=0.92)

plt.text(x, y, f'grobe Hochrechnung: 60% der Bevölkerung geimpft in \nca. {Herdenimmun} Jahren.\nImpfquote Deutschland: {Impfquote_ger}%', 
        horizontalalignment='center', 
        size=size-4, 
        style='italic',
        bbox={'facecolor': c_ger, 'alpha': 0.5, 'pad': 5})

# Diagramm als Bild exporieren und Auflösung definieren
plt.savefig(pfad + name_7, dpi = 150, bbox_inches='tight')

plt.close()
#plt.show()

## RKI - Impfungen Timeseries

In [22]:

def y_axis_thousands(x, pos):
   # 'The two args are the value and tick position'
    return '{:0,d}'.format(int(x)).replace(",",".")
formatter = FuncFormatter(y_axis_thousands)


fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()
ax.yaxis.set_major_formatter(formatter)


ax1 = plt.plot(df_I_TS['Datum'],df_I_TS['MA'], color=c_ger, linestyle = 'solid', linewidth = lwb, label = "Impfungen\n(7-Tage Mittel)") # blau, linie, dick
ax2 = plt.plot(df_I_TS['Datum'],df_I_TS['Gesamtzahl Impfungen'], marker='.', linestyle='', color=c_ger, markersize = 20)

ax3 = plt.plot(df_I_TS['Datum'],df_I_TS['OTG_Impfungen'], color='red', linestyle = 'dashed', linewidth = lws, label ="+- 1 sigma")
ax4 = plt.plot(df_I_TS['Datum'],df_I_TS['UTG_Impfungen'], color='red', linestyle = 'dashed', linewidth = lws, label ="")

ax5 = plt.plot(df_I_TS['Datum'],
               df_I_TS['SOLL_Impfungen60'], 
               color='#34eb46', 
               linestyle = 'dashed', 
               linewidth = lws, 
               label = "SOLL-Impfungen pro Tag\num 15, 30, 60% der Bevölkerung\nbis 01.10.2021 zu impfen")

ax6 = plt.plot(df_I_TS['Datum'],
               df_I_TS['SOLL_Impfungen30'], 
               color='#34eb46', 
               linestyle = 'dashed', 
               linewidth = lws, 
               label = "")

ax7 = plt.plot(df_I_TS['Datum'],
               df_I_TS['SOLL_Impfungen15'], 
               color='#34eb46', 
               linestyle = 'dashed', 
               linewidth = lws, 
               label = "")

# # Schriftgrößen x und y achsenwerte
plt.xticks(fontsize=size - 5, rotation=90)
plt.yticks(fontsize=size - 4)

plt.ylabel('Anzahl Impfungen', fontsize=size)
plt.xlabel('Zeit', fontsize=size)

plt.title('Anzahl Impfungen Deutschland (RKI-Daten)\n', fontsize=size)
plt.suptitle(today + ' PW', fontsize=size-5, y=0.92)

# # fill area between lines
plt.fill_between(df_I_TS['Datum'], df_I_TS['OTG_Impfungen'], df_I_TS['UTG_Impfungen'] , color='red',alpha=0.5)

# Legende 
plt.legend(loc='upper center', 
           bbox_to_anchor=(0.5, -0.3),
           fancybox=True, 
           shadow=True, 
           ncol=2, 
           fontsize=size)

fig.savefig(pfad + name_7_2, dpi = 150, bbox_inches='tight')

plt.close()
#plt.show()



In [23]:

def y_axis_thousands(x, pos):
   # 'The two args are the value and tick position'
    return '{:0,d}'.format(int(x)).replace(",",".")
formatter = FuncFormatter(y_axis_thousands)

fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()
ax.yaxis.set_major_formatter(formatter)

ax1 = plt.plot(df_I_TS['Datum'],df_I_TS['MA'], color=c_ger, linestyle = 'solid', linewidth = lwb, label = "Impfungen\n(7-Tage Mittel)") # blau, linie, dick
ax2 = plt.plot(df_I_TS['Datum'],df_I_TS['Gesamtzahl Impfungen'], marker='.', linestyle='', color=c_ger, markersize = 20)

ax3 = plt.plot(df_I_TS['Datum'],df_I_TS['OTG_Impfungen'], color='red', linestyle = 'dashed', linewidth = lws, label ="+- 1 sigma")
ax4 = plt.plot(df_I_TS['Datum'],df_I_TS['UTG_Impfungen'], color='red', linestyle = 'dashed', linewidth = lws, label ="")

# # Schriftgrößen x und y achsenwerte
plt.xticks(fontsize=size - 5, rotation=90)
plt.yticks(fontsize=size - 4)

plt.ylabel('Anzahl Impfungen', fontsize=size)
plt.xlabel('Zeit', fontsize=size)

plt.title('Anzahl Impfungen Deutschland (RKI-Daten)\n', fontsize=size)
plt.suptitle(today + ' PW', fontsize=size-5, y=0.92)


# # fill area between lines
plt.fill_between(df_I_TS['Datum'], df_I_TS['OTG_Impfungen'], df_I_TS['UTG_Impfungen'] , color='red',alpha=0.5)

# Legende 
plt.legend(loc='upper center', 
           bbox_to_anchor=(0.5, -0.3),
           fancybox=True, 
           shadow=True, 
           ncol=2, 
           fontsize=size)

fig.savefig(pfad + name_7_3, dpi = 150, bbox_inches='tight')

plt.close()
#plt.show()



## dataframes speichern

In [24]:
df_EW.to_csv(r'D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_EW.csv', index=False)
df_I.to_csv(r'D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_I.csv', index=False)
df_I_EW.to_csv(r'D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_I_EW.csv', index=False)
df_I_TS.to_csv(r'D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_I_TS.csv', index=False)

## Performance messen

In [25]:
pc = os.environ['COMPUTERNAME']
now2 = datetime.now()

# Laufzeit
x = now2 - now
x = round(x.total_seconds(),2)

# Performance CSV einlesen
d = pd.read_csv('D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_performance_Führungskreis.csv')

# Neues Dateframe für die Performance definieren
now = datetime.now()

d2 = {'Date': [now],
      'PC': [pc],
      'Laufzeit_in_s': [x],
      'Version': [Programm_Name]}

# Datum Spalte formatieren
df2 = pd.DataFrame(d2)
df2['Date'] = df2['Date'].dt.strftime('%Y-%m-%d %r')

# Performance mit dem CSV verbinden
d = d.append(df2, ignore_index=True)

# Datenexport Performance
d.to_csv(r'D:\\Github\\Covid-19-Auswertung-WHO-DIVI\\output\\Dataframes\\df_performance_Führungskreis.csv', index=False)

print(f'performance {pc} = {x} seconds')

performance DOUG = 1.34 seconds
